In [ ]:
!pip install datasets
!pip install nltk
import re
from datasets import load_dataset
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from collections import Counter
from nltk.util import bigrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

dataset = load_dataset('nid989/EssayFroum-Dataset')

# Access the 'text' field from the dataset
texts = dataset['train']['Correct Grammar']


# Define a function to preprocess the text
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords, punctuation, and quotation marks
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]
    tokens = [word for word in tokens if word not in string.punctuation]
    tokens = [word.replace("``", "").replace("''", "") for word in tokens]

    # Combine contractions like "can't," "won't," "he's," "she's," and "John's" into single words
    combined_tokens = []
    i = 0
    while i < len(tokens):
        if i < len(tokens) - 1 and (tokens[i].endswith("n't") or tokens[i].endswith("'s")):
            combined_tokens.append(tokens[i] + tokens[i + 1])
            i += 2
        else:
            combined_tokens.append(tokens[i])
            i += 1

    return combined_tokens

# Tokenize and preprocess the text
preprocessed_texts = [preprocess_text(text) for text in texts]

# Combine all preprocessed texts into a single list of words
all_words = [word for text in preprocessed_texts for word in text]

# Count unigrams and bigrams
unigram_counts_eng = Counter(all_words)
bigrams_list = list(bigrams(all_words))
bigram_counts_eng = Counter(bigrams_list)

unwanted_unigrams = ["", "--"]
unwanted_bigrams = [('--', '--'), ('', '')]

for unwanted_unigram in unwanted_unigrams:
    if unwanted_unigram in unigram_counts_eng:
        del unigram_counts_eng[unwanted_unigram]

for unwanted_bigram in unwanted_unigrams:
    if unwanted_bigram in unigram_counts_eng:
        del unigram_counts_eng[unwanted_unigram]

for unwanted_bigram in unwanted_bigrams:
    if unwanted_bigram in bigram_counts_eng:
        del bigram_counts_eng[unwanted_bigram]

# Calculate and store the total number of terms and bigrams
total_terms = len(all_words)
total_bigrams = len(bigrams_list)

# Display the top unigrams and bigrams
print("Top Unigrams:")
print(unigram_counts_eng.most_common(10))

print("\nTop Bigrams:")
print(bigram_counts_eng.most_common(10))

# Display the total number of terms and bigrams in the dataset
print(f"Total Number of Terms in the Dataset: {total_terms}")
print(f"Total Number of Bigrams in the Dataset: {total_bigrams}")



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 34.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Top Unigrams:
[('would', 45612), ('life', 36905), ('school', 36653), ('one', 33573), ('time', 27166), ('people', 26979), ('could', 25702), ('like', 24603), ('world', 20679), ('always', 18644)]

Top Bigrams:
[(('high', 'school'), 9706), (('would', 'like'), 2461), (('every', 'day'), 2324), (('first', 'time'), 2248), (('United', 'States'), 2195), (('hard', 'work'), 1943), (('one', 'day'), 1925), (('said', ''), 1835), (('New', 'York'), 1558), (('Even', 'though'), 1553)]
Total Number of Terms in the Dataset: 5903648
Total Number of Bigrams in the Dataset: 5903647


In [ ]:
!pip install PyPDF2
import os
import PyPDF2
from collections import Counter
from nltk.util import bigrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re

# Mount your Google Drive in Google Colab
from google.colab import drive
drive.mount('/content/drive')

# Specify the directory path to the folder containing the PDF files in your Google Drive
pdf_folder_path = '/content/drive/MyDrive/ColabNotebooks/papers/'

# Initialize lists to store unigrams, bigrams, and file names for all papers
all_unigrams = []
all_bigrams = []
file_names = []

# Initialize lists to store total terms and total bigrams for all papers
total_terms_paper = []
total_bigrams_paper = []

# Iterate through the PDF files in the folder
for filename in os.listdir(pdf_folder_path):
    if filename.endswith('.pdf'):
        # Construct the full path to the PDF file
        pdf_file_path = os.path.join(pdf_folder_path, filename)

        # Store the file name
        file_names.append(filename)

        # Open the PDF file and extract text
        with open(pdf_file_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()

        # Tokenize and preprocess the text
        preprocessed_text = preprocess_text(text)

        # Count unigrams and bigrams
        unigram_counts = Counter(preprocessed_text)
        bigram_counts = Counter(bigrams(preprocessed_text))

        # Append the unigrams and bigrams to the respective lists
        all_unigrams.append(list(unigram_counts.keys()))
        all_bigrams.append(list(bigram_counts.keys()))

        # Calculate the total number of terms (unigrams) and the total number of bigrams
        total_terms = len(preprocessed_text)
        total_bigrams = len(list(bigram_counts.keys()))

        # Append the total terms and total bigrams to the respective lists
        total_terms_paper.append(total_terms)
        total_bigrams_paper.append(total_bigrams)

# Now, 'all_unigrams' and 'all_bigrams' are lists of lists containing unigrams and bigrams for all the papers in the folder, and 'file_names' contains the corresponding file names.
# 'total_terms_paper' is a list containing the total number of terms (unigrams) for each paper, and 'total_bigrams_paper' contains the total number of bigrams for each paper.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
#cosine similarity
eng_unigrams = list(unigram_counts_eng.keys())
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize



# Create CountVectorizer to count the term frequency (TF)
vectorizer = CountVectorizer()

results = []

for idx, pdf_unigrams in enumerate(all_unigrams[:9]):
    # Combine the unigrams into text documents
    pdf_text = " ".join(pdf_unigrams)
    other_text = " ".join(eng_unigrams)

    # Create a file name based on the index
    file_name = file_names[idx]

    # Create CountVectorizer and fit it with the text documents
    tf_matrix = vectorizer.fit_transform([pdf_text, other_text])

    # Calculate the cosine similarity between the two vectors
    similarity = cosine_similarity(tf_matrix)

    # Append the results for sorting
    results.append((idx, file_name, similarity[0][1]))

# Sort the results by cosine similarity in descending order
results.sort(key=lambda x: x[2], reverse=True)

# Print the results
for idx, file_name, sim in results:
    print(f"Index: {idx}, File Name: {file_name}, Cosine Similarity: {sim}")


Index: 6, File Name: GPT-3 Language Models are Few-Shot Learners.pdf, Cosine Similarity: 0.28344919627760756
Index: 5, File Name: BERT.pdf, Cosine Similarity: 0.2803297113820578
Index: 4, File Name: A Neural Probabilistic Language Model.pdf, Cosine Similarity: 0.2607868022195515
Index: 2, File Name: GloVe.pdf, Cosine Similarity: 0.25666499101115803
Index: 7, File Name: Skip-Thought Vectors.pdf, Cosine Similarity: 0.21714198380465596
Index: 0, File Name: Attention-is-all-you-need-Paper.pdf, Cosine Similarity: 0.21235935444268964
Index: 8, File Name: Convolutional Neural Networks for Sentence Classification.pdf, Cosine Similarity: 0.20527815239583266
Index: 3, File Name: Sequence to Sequence Learning with Neural Networks.pdf, Cosine Similarity: 0.18844533243499145
Index: 1, File Name: word2vec.pdf, Cosine Similarity: 0.14511713335346849


In [ ]:
#cosine similarity, vectorizer = CountVectorizer(vocabulary=eng_unigrams)
#pdf_unigrams = list(unigram_counts_p.keys())
eng_unigrams = list(unigram_counts_eng.keys())

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize



# Create a CountVectorizer with the vocabulary from the English source
vectorizer = CountVectorizer(vocabulary=eng_unigrams)
results = []

for idx, pdf_unigrams in enumerate(all_unigrams[:9]):
    # Combine the unigrams into text documents
    pdf_text = " ".join(pdf_unigrams)
    other_text = " ".join(eng_unigrams)

    # Calculate the total terms for the current paper
    total_terms_current_paper = total_terms_paper[idx]

    # Calculate the total terms for the English source
    total_terms_eng_source = total_terms

    # Normalize the term frequency values using DLN
    pdf_tf_values = [tf / total_terms_current_paper for tf in vectorizer.transform([pdf_text]).toarray()[0]]
    eng_tf_values = [tf / total_terms_eng_source for tf in vectorizer.transform([other_text]).toarray()[0]]

    # Calculate the cosine similarity between the two vectors
    similarity = cosine_similarity([pdf_tf_values], [eng_tf_values])


    # Append the results for sorting
    results.append((idx, file_names[idx], similarity[0][0]))


    # The 'similarity' variable now contains the cosine similarity score with DLN applied
    #print(f"Index: {idx}, File Name: {file_names[idx]}, Cosine Similarity (with DLN) for {file_names[idx]}: {similarity[0][0]}")
# Sort the results by cosine similarity in descending order
results.sort(key=lambda x: x[2], reverse=True)

# Print the results
for idx, file_name, sim in results:
    print(f"Index: {idx}, Cosine Similarity (with DLN) for {file_name}: {sim}")



Index: 6, Cosine Similarity (with DLN) for GPT-3 Language Models are Few-Shot Learners.pdf: 0.3517325132491673
Index: 2, Cosine Similarity (with DLN) for GloVe.pdf: 0.32150909730635807
Index: 5, Cosine Similarity (with DLN) for BERT.pdf: 0.30876935341591055
Index: 7, Cosine Similarity (with DLN) for Skip-Thought Vectors.pdf: 0.28844541078702923
Index: 4, Cosine Similarity (with DLN) for A Neural Probabilistic Language Model.pdf: 0.2869579316869975
Index: 3, Cosine Similarity (with DLN) for Sequence to Sequence Learning with Neural Networks.pdf: 0.2740048583826688
Index: 0, Cosine Similarity (with DLN) for Attention-is-all-you-need-Paper.pdf: 0.2634061610612729
Index: 8, Cosine Similarity (with DLN) for Convolutional Neural Networks for Sentence Classification.pdf: 0.2421030237527346
Index: 1, Cosine Similarity (with DLN) for word2vec.pdf: 0.22613015380883392


In [ ]:
#cosine similarity, eng and particular paper vocab in vectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize



results = []

for idx, pdf_unigrams in enumerate(all_unigrams[:9]):
    # Create a vocabulary by merging the English source and the current PDF paper's vocabulary
    all_vocabulary = set(eng_unigrams)
    all_vocabulary.update(pdf_unigrams)

    # Create a CountVectorizer with the merged vocabulary
    vectorizer = CountVectorizer(vocabulary=list(all_vocabulary))

    # Combine the unigrams into text documents
    pdf_text = " ".join(pdf_unigrams)
    other_text = " ".join(eng_unigrams)

    # Calculate the total terms for the current paper
    total_terms_current_paper = total_terms_paper[idx]

    # Calculate the total terms for the English source
    total_terms_eng_source = total_terms

    # Fit the vectorizer with the merged vocabulary
    pdf_tf_values = [tf / total_terms_current_paper for tf in vectorizer.transform([pdf_text]).toarray()[0]]
    eng_tf_values = [tf / total_terms_eng_source for tf in vectorizer.transform([other_text]).toarray()[0]]

    # Calculate the cosine similarity
    similarity = cosine_similarity([pdf_tf_values], [eng_tf_values])

    results.append((idx, file_names[idx], similarity[0][0]))

    # The 'similarity' variable now contains the cosine similarity score with DLN applied
    #print(f"Index: {idx}, File Name: {file_names[idx]}, Cosine Similarity (with DLN) for {file_names[idx]}: {similarity[0][0]}")
results.sort(key=lambda x: x[2], reverse=True)

# Print the results
for idx, file_name, sim in results:
    print(f"Index: {idx}, Cosine Similarity (with DLN) for {file_name}: {sim}")


Index: 6, Cosine Similarity (with DLN) for GPT-3 Language Models are Few-Shot Learners.pdf: 0.34810472137937165
Index: 2, Cosine Similarity (with DLN) for GloVe.pdf: 0.30488880500321563
Index: 5, Cosine Similarity (with DLN) for BERT.pdf: 0.2970871653783336
Index: 7, Cosine Similarity (with DLN) for Skip-Thought Vectors.pdf: 0.26853214059736247
Index: 0, Cosine Similarity (with DLN) for Attention-is-all-you-need-Paper.pdf: 0.24991789015066976
Index: 4, Cosine Similarity (with DLN) for A Neural Probabilistic Language Model.pdf: 0.24753458497676747
Index: 8, Cosine Similarity (with DLN) for Convolutional Neural Networks for Sentence Classification.pdf: 0.22549656922479025
Index: 3, Cosine Similarity (with DLN) for Sequence to Sequence Learning with Neural Networks.pdf: 0.2251670899356788
Index: 1, Cosine Similarity (with DLN) for word2vec.pdf: 0.18352082596600172


In [ ]:
#cosine similarity, eng and all pdf as vocab for vectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize


# Create a vocabulary by merging the English source and all PDF papers' vocabularies
all_vocabulary = set(eng_unigrams)
for pdf_unigrams in all_unigrams:
    all_vocabulary.update(pdf_unigrams)

# Create a CountVectorizer with the merged vocabulary
vectorizer = CountVectorizer(vocabulary=list(all_vocabulary))

results = []

for idx, pdf_unigrams in enumerate(all_unigrams[:9]):
    # Combine the unigrams into text documents
    pdf_text = " ".join(pdf_unigrams)
    other_text = " ".join(eng_unigrams)

    # Calculate the total terms for the current paper
    total_terms_current_paper = total_terms_paper[idx]

    # Calculate the total terms for the English source
    total_terms_eng_source = total_terms

    # Fit the vectorizer with the merged vocabulary
    pdf_tf_values = [tf / total_terms_current_paper for tf in vectorizer.transform([pdf_text]).toarray()[0]]
    eng_tf_values = [tf / total_terms_eng_source for tf in vectorizer.transform([other_text]).toarray()[0]]

    # Calculate the cosine similarity
    similarity = cosine_similarity([pdf_tf_values], [eng_tf_values])

    results.append((idx, file_names[idx], similarity[0][0]))



    # The 'similarity' variable now contains the cosine similarity score with DLN applied
    #print(f"Index: {idx}, File Name: {file_names[idx]}, Cosine Similarity (with DLN) for {file_names[idx]}: {similarity[0][0]}")

results.sort(key=lambda x: x[2], reverse=True)

# Print the results
for idx, file_name, sim in results:
    print(f"Index: {idx}, Cosine Similarity (with DLN) for {file_name}: {sim}")


Index: 6, Cosine Similarity (with DLN) for GPT-3 Language Models are Few-Shot Learners.pdf: 0.34769977942185476
Index: 2, Cosine Similarity (with DLN) for GloVe.pdf: 0.30394662215648066
Index: 5, Cosine Similarity (with DLN) for BERT.pdf: 0.2938720217423237
Index: 7, Cosine Similarity (with DLN) for Skip-Thought Vectors.pdf: 0.2676749080374901
Index: 0, Cosine Similarity (with DLN) for Attention-is-all-you-need-Paper.pdf: 0.2511453317414556
Index: 4, Cosine Similarity (with DLN) for A Neural Probabilistic Language Model.pdf: 0.24472619446972324
Index: 3, Cosine Similarity (with DLN) for Sequence to Sequence Learning with Neural Networks.pdf: 0.22464713649078943
Index: 8, Cosine Similarity (with DLN) for Convolutional Neural Networks for Sentence Classification.pdf: 0.22385042754232568
Index: 1, Cosine Similarity (with DLN) for word2vec.pdf: 0.18227897261489034


In [ ]:
#cosine similarity, using words from nltk for vectorizer
import nltk
from nltk.corpus import words
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize

# Download the NLTK English words corpus (if you haven't already)
nltk.download("words")

# Load the list of English words from the NLTK corpus as the general English vocabulary
general_english_vocabulary = set(word.lower() for word in words.words())


# Also, make sure you have 'total_terms_paper' and 'total_terms_eng' calculated

# Create a CountVectorizer with the general English vocabulary
vectorizer = CountVectorizer(vocabulary=general_english_vocabulary)

results = []

for idx, pdf_unigrams in enumerate(all_unigrams[:9]):
    # Combine the unigrams into text documents
    pdf_text = " ".join(pdf_unigrams)
    other_text = " ".join(eng_unigrams)

    # Convert the text documents to lowercase
    pdf_text = pdf_text.lower()
    other_text = other_text.lower()

    # Calculate the total terms for the current paper
    total_terms_current_paper = total_terms_paper[idx]

    # Calculate the total terms for the English source
    total_terms_eng_source = total_terms

    # Normalize the term frequency values using DLN
    pdf_tf_values = [tf / total_terms_current_paper for tf in vectorizer.transform([pdf_text]).toarray()[0]]
    eng_tf_values = [tf / total_terms_eng_source for tf in vectorizer.transform([other_text]).toarray()[0]]

    # Calculate the cosine similarity
    similarity = cosine_similarity([pdf_tf_values], [eng_tf_values])

    results.append((idx, file_names[idx], similarity[0][0]))

    # The 'similarity' variable now contains the cosine similarity score with DLN applied
    #print(f"Index: {idx}, File Name: {file_names[idx]}, Cosine Similarity (with DLN) for {file_names[idx]}: {similarity[0][0]}")

results.sort(key=lambda x: x[2], reverse=True)

# Print the results
for idx, file_name, sim in results:
    print(f"Index: {idx}, Cosine Similarity (with DLN) for {file_name}: {sim}")


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


Index: 6, Cosine Similarity (with DLN) for GPT-3 Language Models are Few-Shot Learners.pdf: 0.42093560494578974
Index: 4, Cosine Similarity (with DLN) for A Neural Probabilistic Language Model.pdf: 0.3361924676641434
Index: 5, Cosine Similarity (with DLN) for BERT.pdf: 0.3159405511579037
Index: 2, Cosine Similarity (with DLN) for GloVe.pdf: 0.2957850169739905
Index: 0, Cosine Similarity (with DLN) for Attention-is-all-you-need-Paper.pdf: 0.25320490299957626
Index: 8, Cosine Similarity (with DLN) for Convolutional Neural Networks for Sentence Classification.pdf: 0.24728287029360377
Index: 7, Cosine Similarity (with DLN) for Skip-Thought Vectors.pdf: 0.24408052986967105
Index: 3, Cosine Similarity (with DLN) for Sequence to Sequence Learning with Neural Networks.pdf: 0.2394003722742422
Index: 1, Cosine Similarity (with DLN) for word2vec.pdf: 0.18618006953368804


In [ ]:
#cosine similarity for bigrams
eng_bigrams = list(bigram_counts_eng.keys())
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import bigrams

results = []

for idx, pdf_bigrams in enumerate(all_bigrams[:9]):
    # Combine the bigrams into text documents as strings
    pdf_text2 = " ".join(" ".join(bigram) for bigram in pdf_bigrams)
    other_text2 = " ".join(" ".join(bigram) for bigram in eng_bigrams)

    # Create a file name based on the index
    file_name = file_names[idx]

    # Create CountVectorizer and fit it with the text documents
    vectorizer = CountVectorizer()
    tf_matrix = vectorizer.fit_transform([pdf_text2, other_text2])

    # Calculate the cosine similarity between the two vectors
    similarity = cosine_similarity(tf_matrix)

    # Append the results for sorting
    results.append((idx, file_name, similarity[0][1]))

# Sort the results by cosine similarity in descending order
results.sort(key=lambda x: x[2], reverse=True)

# Print the results
for idx, file_name, sim in results:
    print(f"Index: {idx}, File Name: {file_name}, Cosine Similarity: {sim}")


Index: 6, File Name: GPT-3 Language Models are Few-Shot Learners.pdf, Cosine Similarity: 0.32165792193260073
Index: 4, File Name: A Neural Probabilistic Language Model.pdf, Cosine Similarity: 0.2604236202557303
Index: 2, File Name: GloVe.pdf, Cosine Similarity: 0.23219224795860932
Index: 7, File Name: Skip-Thought Vectors.pdf, Cosine Similarity: 0.23161715666763327
Index: 3, File Name: Sequence to Sequence Learning with Neural Networks.pdf, Cosine Similarity: 0.20173671161592874
Index: 0, File Name: Attention-is-all-you-need-Paper.pdf, Cosine Similarity: 0.1960848052236799
Index: 5, File Name: BERT.pdf, Cosine Similarity: 0.19186117238466893
Index: 1, File Name: word2vec.pdf, Cosine Similarity: 0.1805514986145058
Index: 8, File Name: Convolutional Neural Networks for Sentence Classification.pdf, Cosine Similarity: 0.1685540766115835


In [ ]:
#using tfid
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



results = []

for idx, pdf_unigrams in enumerate(all_unigrams[:9]):
    # Combine the unigrams into text documents
    pdf_text = " ".join(pdf_unigrams)
    other_text = " ".join(eng_unigrams)

    # Create a file name based on the index
    file_name = file_names[idx]

    # Create TfidfVectorizer
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer with the text documents
    tfidf_matrix = vectorizer.fit_transform([pdf_text, other_text])

    # Calculate the cosine similarity between the two vectors
    similarity = cosine_similarity(tfidf_matrix)

    # Append the results for sorting
    results.append((idx, file_name, similarity[0][1]))

# Sort the results by cosine similarity in descending order
results.sort(key=lambda x: x[2], reverse=True)

# Print the results
for idx, file_name, sim in results:
    print(f"Index: {idx}, File Name: {file_name}, Cosine Similarity: {sim}")


Index: 6, File Name: GPT-3 Language Models are Few-Shot Learners.pdf, Cosine Similarity: 0.23707529227644733
Index: 5, File Name: BERT.pdf, Cosine Similarity: 0.21873748859480388
Index: 4, File Name: A Neural Probabilistic Language Model.pdf, Cosine Similarity: 0.192769827483206
Index: 2, File Name: GloVe.pdf, Cosine Similarity: 0.18728060674621858
Index: 7, File Name: Skip-Thought Vectors.pdf, Cosine Similarity: 0.16030611289873462
Index: 0, File Name: Attention-is-all-you-need-Paper.pdf, Cosine Similarity: 0.1508590217107586
Index: 8, File Name: Convolutional Neural Networks for Sentence Classification.pdf, Cosine Similarity: 0.14818399218866712
Index: 3, File Name: Sequence to Sequence Learning with Neural Networks.pdf, Cosine Similarity: 0.12087884237468413
Index: 1, File Name: word2vec.pdf, Cosine Similarity: 0.0943349246239318


In [ ]:
#jaccard similarity
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

results = []

for idx, pdf_unigrams in enumerate(all_unigrams[:9]):
    pdf_set = set(pdf_unigrams)
    eng_set = set(eng_unigrams)

    # Calculate the Jaccard similarity between the two sets
    jaccard_sim = jaccard_similarity(pdf_set, eng_set)

    # Append the results for sorting
    results.append((idx, file_names[idx], jaccard_sim))

# Sort the results by Jaccard similarity in descending order
results.sort(key=lambda x: x[2], reverse=True)

# Print the results
for idx, file_name, sim in results:
    print(f"Index: {idx}, File Name: {file_name}, Jaccard Similarity: {sim}")


Index: 6, File Name: GPT-3 Language Models are Few-Shot Learners.pdf, Jaccard Similarity: 0.03753894972724968
Index: 5, File Name: BERT.pdf, Jaccard Similarity: 0.011820370216651449
Index: 7, File Name: Skip-Thought Vectors.pdf, Jaccard Similarity: 0.011178023772041613
Index: 2, File Name: GloVe.pdf, Jaccard Similarity: 0.011069895871452809
Index: 4, File Name: A Neural Probabilistic Language Model.pdf, Jaccard Similarity: 0.009896222498962225
Index: 0, File Name: Attention-is-all-you-need-Paper.pdf, Jaccard Similarity: 0.008328604332877123
Index: 3, File Name: Sequence to Sequence Learning with Neural Networks.pdf, Jaccard Similarity: 0.0065752285951787195
Index: 8, File Name: Convolutional Neural Networks for Sentence Classification.pdf, Jaccard Similarity: 0.006115417024720301
Index: 1, File Name: word2vec.pdf, Jaccard Similarity: 0.006070383182543907


In [ ]:
#jaccard similarity with dln
from sklearn.metrics import jaccard_score

results = []

for idx, pdf_unigrams in enumerate(all_unigrams[:9]):
    # Create a set of unigrams for the PDF document
    pdf_set = set(pdf_unigrams)

    # Create a set of unigrams for the English source
    eng_set = set(eng_unigrams)

    # Calculate the Jaccard similarity between the two sets
    jaccard_sim = len(pdf_set.intersection(eng_set)) / len(pdf_set.union(eng_set))

    # Apply Document Length Normalization (DLN)
    pdf_dln = len(pdf_set) / total_terms_paper[idx]
    eng_dln = len(eng_set) / total_terms_eng_source

    jaccard_sim_dln = jaccard_sim * pdf_dln * eng_dln

    # Append the results for sorting
    results.append((idx, file_names[idx], jaccard_sim_dln))

# Sort the results by Jaccard similarity with DLN in descending order
results.sort(key=lambda x: x[2], reverse=True)

# Print the results
for idx, file_name, sim in results:
    print(f"Index: {idx}, File Name: {file_name}, Jaccard Similarity (with DLN): {sim}")


Index: 6, File Name: GPT-3 Language Models are Few-Shot Learners.pdf, Jaccard Similarity (with DLN): 0.5249068256298535
Index: 7, File Name: Skip-Thought Vectors.pdf, Jaccard Similarity (with DLN): 0.25339821429017073
Index: 2, File Name: GloVe.pdf, Jaccard Similarity (with DLN): 0.24041778221538304
Index: 4, File Name: A Neural Probabilistic Language Model.pdf, Jaccard Similarity (with DLN): 0.20980846926766458
Index: 5, File Name: BERT.pdf, Jaccard Similarity (with DLN): 0.20400454530228238
Index: 1, File Name: word2vec.pdf, Jaccard Similarity (with DLN): 0.18320725992908632
Index: 0, File Name: Attention-is-all-you-need-Paper.pdf, Jaccard Similarity (with DLN): 0.181891113358821
Index: 3, File Name: Sequence to Sequence Learning with Neural Networks.pdf, Jaccard Similarity (with DLN): 0.1741544228357791
Index: 8, File Name: Convolutional Neural Networks for Sentence Classification.pdf, Jaccard Similarity (with DLN): 0.14836752589125785
